In [1]:
import glob

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

training_data_path = r"/home/manny/PycharmProjects/TweetAnalysis/florida/training_data"
results_path = r"/home/manny/PycharmProjects/TweetAnalysis/florida/results"

tweets = []
labels_list = []

tweetlabel_dict = {}

# pick file: media, utility, gov, or nonprofit
# typeoffile = sys.argv[1]
typeoffile = 'utility'

file_name = ''

print("Loading data")
test = pd.read_csv(
    '/home/manny/PycharmProjects/TweetAnalysis/florida/training_data/supervised_data/utility/combined.csv')
test = test[test['Manual Coding'] != 7]
test = test[['Tweet', 'Manual Coding', 'Date', 'Permalink']]

X_train, X_test, y_train, y_test = train_test_split(test['Tweet'], test['Manual Coding'])

# Loading Corpus to train tfidf on
print("Loading corpus data")

corpus = pd.read_csv('/home/manny/PycharmProjects/TweetAnalysis/src/corpus.csv')
corpus = corpus.applymap(str)
corpus = corpus['Tweet'].fillna("  ")

# fit tf-idf with the training data
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2),
                        stop_words='english')

# Training tfidf on corpus
# tfidf.fit(frame['Tweet'])
print("fitting corpus")

tfidf.fit(corpus)
# tfidf.fit(test['Tweet'])


/home/manny/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/manny/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Loading data
Loading corpus data
fitting corpus


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='latin-1', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [5]:

# train the model    X_train, X_test, y_train, y_test
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [35]:

sm = SMOTE( sampling_strategy="not majority")
X_train_sm, y_train_sm = sm.fit_sample(X_train_tfidf, y_train)



ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 6

In [14]:
# xtrain, xtest, ytrain, ytest = train_test_split(x, y)
print("training rf model")
clf = RandomForestClassifier(max_features='sqrt', n_estimators=40, n_jobs=-1, verbose=2).fit(X_train_sm, y_train_sm)

print("SCORING rf model")

print(clf.score(tfidf.transform(X_test), y_test))


training rf model
building tree 1 of 40
building tree 2 of 40
building tree 3 of 40
building tree 4 of 40


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 5 of 40
building tree 6 of 40
building tree 7 of 40
building tree 8 of 40
building tree 9 of 40
building tree 10 of 40
building tree 11 of 40
building tree 12 of 40
building tree 13 of 40
building tree 14 of 40
building tree 15 of 40
building tree 16 of 40
building tree 17 of 40
building tree 18 of 40
building tree 19 of 40
building tree 20 of 40
building tree 21 of 40
building tree 22 of 40
building tree 23 of 40
building tree 24 of 40
building tree 25 of 40
building tree 26 of 40
building tree 27 of 40
building tree 28 of 40
building tree 29 of 40
building tree 30 of 40
building tree 31 of 40
building tree 32 of 40
building tree 33 of 40
building tree 34 of 40
building tree 35 of 40
building tree 36 of 40


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   33.2s


building tree 37 of 40
building tree 38 of 40
building tree 39 of 40
building tree 40 of 40


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   38.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:    0.0s finished


SCORING rf model
0.5652173913043478


In [25]:
from sklearn.metrics import f1_score
print(f1_score(y_test, clf.predict(tfidf.transform(X_test)), average=None))

[0.54545455 0.44871795 0.67935578 0.         0.         0.
 0.         0.         0.         0.3902439  0.27272727 0.24
 0.68852459 0.30927835]


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:    0.0s finished
/home/manny/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
